In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory

2025-06-26 17:23:04.638821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750958584.846356      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750958584.912003      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Importing Dataset

In [2]:
!pip install -U gdown
import gdown
# https://drive.google.com/file/d/1vykBCm95woLZc8dLy4ARURIysPatI9TH/view?usp=sharing

file_id = "1vykBCm95woLZc8dLy4ARURIysPatI9TH"  # Replace with your actual file ID
url = f"https://drive.google.com/uc?id={file_id}"
output = "image_dataset.zip"  # Choose your desired output filename

gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1vykBCm95woLZc8dLy4ARURIysPatI9TH
From (redirected): https://drive.google.com/uc?id=1vykBCm95woLZc8dLy4ARURIysPatI9TH&confirm=t&uuid=9b0061ad-783a-4315-8898-2784a7bccef9
To: /kaggle/working/image_dataset.zip
100%|██████████| 2.90G/2.90G [00:34<00:00, 83.4MB/s]


'image_dataset.zip'

In [3]:
import zipfile
import os

# Path to our zip file
zip_path = "/kaggle/working/image_dataset.zip"

# Destination directory
extract_dir = "/kaggle/working/extracted_files"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction completed.")


Extraction completed.


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
TRAIN_PATH = "/kaggle/working/extracted_files/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
VAL_PATH = "/kaggle/working/extracted_files/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

# Loading datasets
train_ds = image_dataset_from_directory(
    TRAIN_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    VAL_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)


Found 70295 files belonging to 38 classes.


I0000 00:00:1750958678.157159      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 17572 files belonging to 38 classes.


## Model implementation

In [5]:

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])


In [6]:
import numpy as np
class_counts = np.bincount(np.concatenate([y for x, y in train_ds]))
print(dict(zip(class_names, class_counts)))


{'Apple___Apple_scab': 2016, 'Apple___Black_rot': 1987, 'Apple___Cedar_apple_rust': 1760, 'Apple___healthy': 2008, 'Blueberry___healthy': 1816, 'Cherry_(including_sour)___Powdery_mildew': 1683, 'Cherry_(including_sour)___healthy': 1826, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 1642, 'Corn_(maize)___Common_rust_': 1907, 'Corn_(maize)___Northern_Leaf_Blight': 1908, 'Corn_(maize)___healthy': 1859, 'Grape___Black_rot': 1888, 'Grape___Esca_(Black_Measles)': 1920, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 1722, 'Grape___healthy': 1692, 'Orange___Haunglongbing_(Citrus_greening)': 2010, 'Peach___Bacterial_spot': 1838, 'Peach___healthy': 1728, 'Pepper,_bell___Bacterial_spot': 1913, 'Pepper,_bell___healthy': 1988, 'Potato___Early_blight': 1939, 'Potato___Late_blight': 1939, 'Potato___healthy': 1824, 'Raspberry___healthy': 1781, 'Soybean___healthy': 2022, 'Squash___Powdery_mildew': 1736, 'Strawberry___Leaf_scorch': 1774, 'Strawberry___healthy': 1824, 'Tomato___Bacterial_spot': 17

In [7]:
# Loading the base model - EfficientNetB0
base_model = EfficientNetB0(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True  # Unfreezing for fine-tuning

# Final Model
model = Sequential([
    data_augmentation,
    tf.keras.layers.Lambda(preprocess_input),   # Correct preprocessing for EfficientNet
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(NUM_CLASSES, activation='softmax', dtype='float32')  # float32 for mixed precision support
])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
# Compile with lower LR for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

# Training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10


E0000 00:00:1750958768.454766      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1750958773.790993     105 cuda_dnn.cc:529] Loaded cuDNN version 90300


2197/2197 ━━━━━━━━━━━━━━━━━━━━ 654s 271ms/step - accuracy: 0.3472 - loss: 2.7729 - val_accuracy: 0.8998 - val_loss: 0.5045
Epoch 2/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 591s 269ms/step - accuracy: 0.9040 - loss: 0.4396 - val_accuracy: 0.9547 - val_loss: 0.1690
Epoch 3/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 592s 269ms/step - accuracy: 0.9532 - loss: 0.1853 - val_accuracy: 0.9693 - val_loss: 0.1013
Epoch 4/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 592s 269ms/step - accuracy: 0.9688 - loss: 0.1130 - val_accuracy: 0.9768 - val_loss: 0.0746
Epoch 5/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 592s 269ms/step - accuracy: 0.9781 - loss: 0.0787 - val_accuracy: 0.9810 - val_loss: 0.0598
Epoch 6/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 592s 270ms/step - accuracy: 0.9836 - loss: 0.0587 - val_accuracy: 0.9839 - val_loss: 0.0491
Epoch 7/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 593s 270ms/step - accuracy: 0.9874 - loss: 0.0453 - val_accuracy: 0.9845 - val_loss: 0.0445
Epoch 8/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 595s 271ms/step - accuracy: 0.9

## Model Testing

In [9]:
test_ds = image_dataset_from_directory(
    "/kaggle/working/extracted_files/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
model.evaluate(test_ds)


Found 17572 files belonging to 38 classes.
550/550 ━━━━━━━━━━━━━━━━━━━━ 27s 49ms/step - accuracy: 0.9931 - loss: 0.0203


[0.032943230122327805, 0.9889028072357178]

In [10]:
model.save("plant_disease_model.keras")


<!-- On test data -->

In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/kaggle/working/extracted_files/test/test/AppleScab1.JPG"
img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # shape (1, 224, 224, 3)

# Preprocessing using EfficientNet's expected format
img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)

# Prediction
predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]
confidence = np.max(predictions)

print(f"Predicted: {predicted_class} ({confidence:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted: Apple___Apple_scab (1.00)


In [12]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input

def predict_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".JPG"):
            img_path = os.path.join(folder_path, filename)
            img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
            img_array = tf.keras.utils.img_to_array(img)
            img_array = np.expand_dims(img_array, 0)
            img_array = preprocess_input(img_array)

            predictions = model.predict(img_array)
            pred_label = class_names[np.argmax(predictions)]
            confidence = np.max(predictions)

            print(f"{filename} --> Predicted: {pred_label} ({confidence:.2f})")


In [13]:
predict_folder("/kaggle/working/extracted_files/test/test")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
TomatoEarlyBlight3.JPG --> Predicted: Tomato___Early_blight (0.78)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
AppleScab2.JPG --> Predicted: Apple___Apple_scab (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
PotatoEarlyBlight5.JPG --> Predicted: Potato___Early_blight (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
TomatoHealthy4.JPG --> Predicted: Tomato___healthy (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
TomatoYellowCurlVirus2.JPG --> Predicted: Tomato___Tomato_Yellow_Leaf_Curl_Virus (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
TomatoEarlyBlight4.JPG --> Predicted: Tomato___Early_blight (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
AppleCedarRust4.JPG --> Predicted: Apple___Cedar_apple_rust (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
PotatoEarlyBlight3.JPG --> Predicted: Potato___Early_blight (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
AppleCedarRust2.JPG --> Predicted: Apple___Cedar_apple_rust (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
PotatoEarlyBligh

## Saving the model -  to be used in Streamlit web app

In [14]:
model.save('/kaggle/working/plant_disease_model.h5')


In [15]:
# from tensorflow.keras.models import load_model

# model = load_model('/kaggle/working/best_model.keras')

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 38)                  │          48,678 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,210,703 (46.58 MB)

 Trainable params: 4,056,226 (15.47 MB)

 Non-trainable params: 42,023 (164.16 KB)

 Optimizer params: 8,112,454 (30.95 MB)